In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt 
import netCDF4 as nc
from netCDF4 import Dataset
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from pyproj import CRS, Transformer
import xarray as xr

In [2]:
#Importing bottom features 

EMOD_features_filepath = "/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/features/ready-for-training/EMOD_features.nc"
EMOD_features = xr.open_dataset(EMOD_features_filepath)

SINMOD_bottom_features_filepath = "/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/features/ready-for-training/SINMOD_bottom_features.nc"
SINMOD_bottom_features = xr.open_dataset(SINMOD_bottom_features_filepath)

In [3]:
print(list(SINMOD_bottom_features.variables))

['stat', 'spatial_ref', 'x', 'y', 'bottom_temperature_features', 'bottom_salinity_features', 'bottom_current_features', 'bottom_statistical_northness_features', 'bottom_statistical_eastness_features', 'current_aspect_angle']


In [4]:
mean = SINMOD_bottom_features.sel(stat='mean')
minimum = SINMOD_bottom_features.sel(stat='10th_percentile')
maximum = SINMOD_bottom_features.sel(stat='90th_percentile')

mean_data_df = mean.to_dataframe().reset_index()
min_data_df = minimum.to_dataframe().reset_index()
max_data_df = maximum.to_dataframe().reset_index()

mean_data_df = mean_data_df.rename(columns={col: f"{col}_mean" for col in mean_data_df.columns if col not in ['x', 'y']})
min_data_df = min_data_df.rename(columns={col: f"{col}_10th_percentile" for col in min_data_df.columns if col not in ['x', 'y']})
max_data_df = max_data_df.rename(columns={col: f"{col}_90th_percentile" for col in max_data_df.columns if col not in ['x', 'y']})

In [5]:
env_data_df = SINMOD_bottom_features[['x', 'y', 'bottom_temperature_features', 'bottom_salinity_features',
                                  'bottom_current_features', 'bottom_statistical_northness_features',
                                  'bottom_statistical_eastness_features', 'current_aspect_angle']].to_dataframe().reset_index()

print(env_data_df.shape)

(241533225, 9)


In [6]:
final_df = env_data_df.merge(mean_data_df, on=['x', 'y'], suffixes=('', '_mean'))


In [ ]:
final_df = final_df.merge(min_data_df, on=['x', 'y'], suffixes=('_mean', '_10th_percentile'))

In [ ]:
final_df = final_df.merge(max_data_df, on=['x', 'y'], suffixes=('_10th_percentile', '_90th_percentile'))

In [5]:
env_data_df_no_nan = final_df.dropna().copy()
print(env_df_no_nan.shape)

(100128072, 9)


In [ ]:
print(env_data_df_no_nan.cols)

In [6]:
scaler = StandardScaler()
features_to_scale = [
    'bottom_temperature_features',
    'bottom_salinity_features',
    'bottom_current_features',
    'bottom_statistical_northness_features',
    'bottom_statistical_eastness_features',
    'current_aspect_angle'
]
scaled_env_data = scaler.fit_transform(env_data_df_no_nan[features_to_scale])
env_data_df_no_nan[features_to_scale] = scaled_env_data



In [7]:
temp_10th_percentile = np.percentile(env_data_df_no_nan['bottom_temperature_features'], 10)
temp_90th_percentile = np.percentile(env_data_df_no_nan['bottom_temperature_features'], 90)


In [8]:
print(env_data_df_no_nan['bottom_temperature_features'].shape)

(100128072,)


In [9]:
print(SINMOD_bottom_features.variables['stat'])

<xarray.IndexVariable 'stat' (stat: 3)> Size: 180B
array(['mean', '10th_percentile', '90th_percentile'], dtype='<U15')


In [10]:
mean_data = SINMOD_bottom_features.sel(stat='mean')
print(mean_data)

<xarray.Dataset> Size: 3GB
Dimensions:                                (x: 8325, y: 9671)
Coordinates:
    stat                                   <U15 60B 'mean'
  * x                                      (x) float64 67kB -6.546e+04 ... 6....
  * y                                      (y) float64 77kB 7.539e+06 ... 6.6...
Data variables:
    spatial_ref                            int64 8B ...
    bottom_temperature_features            (y, x) float32 322MB nan nan ... nan
    bottom_salinity_features               (y, x) float32 322MB nan nan ... nan
    bottom_current_features                (y, x) float32 322MB nan nan ... nan
    bottom_statistical_northness_features  (y, x) float64 644MB nan nan ... nan
    bottom_statistical_eastness_features   (y, x) float64 644MB nan nan ... nan
    current_aspect_angle                   (y, x) float64 644MB nan nan ... nan
